In [1]:
import sys
sys.path.append('/oak/stanford/groups/rbaltman/luyang/scripts/POPDx/code/')
%load_ext autoreload
%autoreload 2

### ICD-10 embeddings

In [2]:
from embeddings import *
import numpy as np
import json

In [13]:
data = pd.read_csv("icd_meaning.csv", encoding='utf-8')["meaning"]
with open('icd_meaning.txt','a+', encoding='utf-8') as f:
    for line in data.values:
        f.write((line)+'\n')

In [3]:
import transformers
print(transformers.__version__)

4.26.1


In [3]:
l2i, i2l, onto_embeddings = ontology_emb(dim=500, ICD_network_file = '../data/19.csv', save_dir = './embeddings/', use_pretrain = True)

No pretrained ONTOLOGY embeddings found.
> /home/shared_data/zdukb/POPDx/code/embeddings.py(65)ontology_emb()
     64     ipdb.set_trace()
---> 65     lset = np.sort(list(lset))
     66     nl = len(lset)

Num of relationships: 19155
Num edges: 19156
> /home/shared_data/zdukb/POPDx/code/embeddings.py(89)ontology_emb()
     88     ipdb.set_trace()
---> 89     sp = graph_shortest_path(A,method='FW',directed=False)
     90 



/home/yuis/.conda/envs/ukb_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function graph_shortest_path is deprecated; `graph_shortest_path` is deprecated in 1.0 (renaming of 0.25) and will be removed in 1.2. Use `scipy.sparse.csgraph.shortest_path` instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
biboert_embeddings, biboert_embeddings_dict = run_bert(use_pretrain = False)

cuda:0


Some weights of the model checkpoint at monologg/biobert_v1.0_pubmed_pmc were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batch # 1
Batch # 2
Batch # 3
Batch # 4
Batch # 5
Batch # 6
Batch # 7
Batch # 8
Batch # 9
Batch # 10
Batch # 11
Batch # 12
Batch # 13
Batch # 14
Batch # 15
Batch # 16
Batch # 17
Batch # 18
Batch # 19
Batch # 20
Batch # 21
Batch # 22
Batch # 23
Batch # 24
Batch # 25
Batch # 26
Batch # 27
Batch # 28
Batch # 29
Batch # 30
Batch # 31
Batch # 32
Batch # 33
Batch # 34
Batch # 35
Batch # 36
Batch # 37
Batch # 38
Batch # 39
Batch # 40
Batch # 41
Batch # 42
Batch # 43
Batch # 44
Batch # 45
Batch # 46
Batch # 47
Batch # 48
Batch # 49
Batch # 50
Batch # 51
Batch # 52
Batch # 53
Batch # 54
Batch # 55
Batch # 56
Batch # 57
Batch # 58
Batch # 59
Batch # 60
Batch # 61
Batch # 62
Batch # 63
Batch # 64
Batch # 65
Batch # 66
Batch # 67
Batch # 68
Batch # 69
Batch # 70
Batch # 71
Batch # 72
Batch # 73
Batch # 74
Batch # 75
Batch # 76
Batch # 77
Batch # 78
Batch # 79
Batch # 80
Batch # 81
Batch # 82
Batch # 83
Batch # 84
Batch # 85
Batch # 86
Batch # 87
Batch # 88
Batch # 89
Batch # 90
Batch # 91
Batch # 

In [3]:
onto_embeddings = np.load("./embeddings/SVD_embedding_dim500.npy")

In [4]:
with open("./icd_embeddings/embedding_node_to_idx_dict.json", 'r') as inpt:
    l2i = json.load(inpt)
# with open('../data/mc_icd10_labels.txt','r') as f:
#     labels = f.readlines()
labels = pd.read_csv("/home/shared_data/zdukb/POPDx/data/icd10_meaning.csv")["ICD10"].tolist()
# labels = [x.strip() for x in labels] 
labels_idx = [l2i[l] for l in labels]
onto_embeddings = onto_embeddings[labels_idx, :]
print(onto_embeddings.shape)

(12274, 500)


In [5]:
biboert_embeddings = np.load("/home/shared_data/zdukb/POPDx/code/icd_text_embeddings/biobert_embeddings.npy")
biboert_embeddings.shape

(12274, 768)

In [6]:
Y_emb_concat = np.concatenate((onto_embeddings, biboert_embeddings), axis=1)
print(Y_emb_concat.shape)

(12274, 1268)


In [23]:
np.save('../data/icd10_label_embed.npy', Y_emb_concat)

### Phecode embeddings


In [9]:
df_phecode = pd.read_csv('../data/Phecode_map_icd10.csv')

In [25]:
df_phecode.head()

,ICD10,PHECODE,Exl. Phecodes,Excl. Phenotypes
0,A009,8.0,001-009.99,Intestinal infection
1,A010,8.5,001-009.99,Intestinal infection
2,A011,8.0,001-009.99,Intestinal infection
3,A012,8.0,001-009.99,Intestinal infection
4,A013,8.0,001-009.99,Intestinal infection


In [10]:
with open('../data/phecode_labels.txt', 'r') as f:
    phecode_labels = f.readlines() 
phecode_labels = [float(x.strip()) for x in phecode_labels] 

In [5]:
phecode_labels = pd.read_csv("/home/shared_data/zdukb/COVID-19/process_5_4/phecode_csv.csv")["PHECODE"].tolist()

In [22]:
phecode_labels

[8.0,
 8.5,
 8.51,
 8.52,
 8.6,
 8.7,
 10.0,
 31.0,
 31.1,
 38.0,
 38.1,
 38.2,
 41.0,
 41.1,
 41.2,
 41.21,
 41.4,
 53.0,
 53.1,
 54.0,
 70.0,
 70.1,
 70.2,
 70.3,
 71.1,
 78.0,
 79.0,
 79.1,
 79.2,
 90.0,
 90.2,
 90.3,
 110.1,
 110.11,
 110.12,
 110.13,
 110.2,
 112.0,
 112.3,
 117.0,
 117.1,
 117.2,
 117.3,
 117.4,
 130.0,
 130.1,
 131.0,
 132.0,
 132.1,
 133.0,
 134.0,
 134.1,
 136.0,
 145.0,
 145.1,
 145.2,
 145.3,
 145.4,
 145.5,
 149.0,
 149.1,
 149.2,
 149.3,
 149.4,
 149.9,
 150.0,
 151.0,
 153.0,
 153.2,
 153.3,
 155.0,
 155.1,
 157.0,
 158.0,
 159.0,
 159.2,
 159.3,
 159.4,
 164.0,
 165.0,
 165.1,
 170.1,
 170.2,
 172.11,
 172.2,
 172.3,
 173.0,
 174.1,
 174.11,
 174.3,
 180.1,
 180.3,
 182.0,
 184.0,
 184.1,
 184.11,
 184.2,
 185.0,
 187.0,
 187.1,
 187.2,
 187.8,
 189.0,
 189.11,
 189.12,
 189.2,
 189.21,
 189.4,
 190.0,
 191.0,
 191.1,
 191.11,
 193.0,
 194.0,
 195.0,
 195.1,
 195.3,
 198.0,
 198.1,
 198.2,
 198.3,
 198.4,
 198.5,
 198.6,
 198.7,
 199.0,
 200.0,
 200.1,
 

In [18]:
df_phecode = pd.read_csv("../data/phecode_icd10_new.csv")

In [19]:
df_phecode

,Unnamed: 0.1,Unnamed: 0,ALT_CODE,ICD10,PHECODE,Excl. Phecodes,ICD10 String,Phenotype,Excl. Phenotypes,ICD10_NOX
0,0,0,A00,A00,8.00,001-009.99,Cholera,Intestinal infection,Intestinal infection,A00
1,1,1,A000,A00.0,8.00,001-009.99,"Cholera due to Vibrio cholerae 01, biovar chol...",Intestinal infection,Intestinal infection,A000
2,2,2,A001,A00.1,8.00,001-009.99,"Cholera due to Vibrio cholerae 01, biovar eltor",Intestinal infection,Intestinal infection,A001
3,3,3,A009,A00.9,8.00,001-009.99,"Cholera, unspecified",Intestinal infection,Intestinal infection,A009
4,4,4,A01,A01,8.00,001-009.99,Typhoid and paratyphoid fevers,Intestinal infection,Intestinal infection,A01
...,...,...,...,...,...,...,...,...,...,...
12865,12865,12865,Z973,Z97.3,367.00,367-367.99,Presence of spectacles and contact lenses,Disorders of refraction and accommodation; bli...,Blindness and low vision,Z973
12866,12866,12866,Z974,Z97.4,389.00,387-389.99,Presence of external hearing-aid,Hearing loss,Hearing loss and related disorders,Z974
12867,12867,12867,Z991,Z99.1,509.80,500-509.99,Dependence on respirator,Dependence on respirator [Ventilator] or suppl...,Pneumoconioses and other lung diseases,Z991
12868,12868,12868,Z992,Z99.2,585.31,580-590.99,Dependence on renal dialysis,Renal dialysis,diseases of kidney and ureters,Z992


In [10]:
phecode_labels_emb = np.zeros((1568, 1268))
for i,phelabel in enumerate(phecode_labels):
    #print(phelabel)
    corr_icd10 = df_phecode.loc[df_phecode.PheCode==phelabel].ALT_CODE_coding19.tolist()
    corr_idx = []
    for l in corr_icd10:
        corr_idx.append(labels.index(l))
    assert (corr_icd10 == np.array(labels)[corr_idx]).all()
    phecode_labels_emb[i,:] = Y_emb_concat[corr_idx, :].mean(axis=0)

In [11]:
phecode_labels_emb.shape

(1568, 1268)

In [3]:
biobert_embed = np.load("./data_phe_text/biobert_embeddings.npy")
data = np.load("./data_phe_text/biobert_embeddings_dict.npz")
be_dict = dict(zip(data, (data[k] for k in data)))
be_dict_list = list(be_dict.keys())

In [6]:
phecode_df_emb = np.zeros((1568, 768))
for i,phelabel in enumerate(phecode_labels):
    corr_idx = be_dict_list.index(str(phelabel))
    phecode_df_emb[i,:] = biobert_embed[corr_idx,:]

In [7]:
phecode_df_emb.shape

(1568, 768)

In [8]:
phe_label_emb = np.load("../data/data_icd_text/phe_label_embed.npy")#(1568,1268)前500是onto
phe_label_emb_phedf =  np.concatenate([phe_label_emb,phecode_df_emb],axis=1)